# Descrição experimentos usando NETNS (isolamento de interfaces usando ambientes de rede virtualizados - namespaces)

- Objetivo configurar um script para automatizar tudo.

- Ref ( muito bom inclusive ):  https://www.youtube.com/watch?v=jhk7y7mYI6s&ab_channel=TechUpskill

# COMO RODAR O SIMULADOR DE REDE ?

- execute `sh setup_5dominios.sh`

- Vão executar um controlador para cada domínio, um host (cada um um terminal) e um switch (nao abre terminal = parent namespace)

- os hosts são management hosts, ou seja, participam da blockchain, escutando por FREDS enviados pelo controlador (abre um terminal)

- e são também, hosts convencionais, que possuem o daemon (não tão daemon) que escuta pacotes de monitoramento (abre outro terminal)

# COMO FINALIZAR ?

- execute `sh end_experiments.sh`


- em cada terminal de controlador execute: `sh run_flowpri2.sh`

- pronto


# Para fazer os testes de overhead = combinar iftop, iperf, tcpdump e logs (sh run_flowpri2.sh > controller.log)

- analise os logs dos controladores e os tempos de saida (tcpdump nas interfaces dos hosts)

- execute um servidor em um host `iperf -s (-u se quiser udp)`

- e em outro host, um client `iperf -c 172.16.x.x (-u se for udp)`


# Para os testes de desempenho de fluxos BE vs QoS:

- execute um servidor em um host `iperf -s -u se quiser udp`

- e em outro host, um client `iperf -c 172.16.x.x -u`

- entao, execute o streaming de video para o par..asd.sad.asd.

- observar as métricas....


# Experimentos:
(so tenho tempo para 5 rodadas de cada -- média e desvio padrão)

* Experimento 1: PingSemFlowpri == Tirar os tempos para verificar o atraso do enlace (1..5 dominios)

* Experimento 2: Tempo primeiro pacote fluxo até chegar no destino

* Experimento 3: Tempo entre o primeiro pacote e a classificacao (10 pacotes)

* Experimento 4:  Tempo entre o primeiro pacote e todos os dominios criarem as regras de qos (10 pacotes)

* Experimento 5: Tempo entre a criação da regra de monitoramento até o ultimo host monitorar e enviar a transação

* Experimento 6: Tempo de setup das blockchains (duração e o tempo entre a primeira e a ultima)

* Experimento 7: Qtd pacotes utilizadas pela blockchain


# Verificar saida de um container:

- execute
`sudo docker logs --tail=350 9d4252e6e83c`



# Algumas informações sobre os comandos do scritp Criar um namespace para cada domínio.

<!-- sudo ip link add <nome-interface> type dummy # isso nao funcionou direito, infelizmente -->

* listar namespaces ativos:
    `sudo ip netns list`

* Criar namespace:
    `sudo ip netns add VRF1`    

* Criando interfaces para utilizar com os namespaces
    `sudo ip link add veth1 type veth peer name veth2`


* Mover interface para namespace:
    `sudo ip link set veth1 netns VRF1`


* Executar comando a partir da namespace (ex: ping, /bin/bash, ifconfig):
    `sudo ip netns exec VRF1 <comando>`

* Subir as interfaces dentro dos namespaces:
    `sudo ip netns exec VRF1 ifconfig veth1 up`

* Criar ovs bridge:
    `sudo ovs-vsctl add-br switch1`

* Adicionar as interfaces criadas:
    `sudo ovs-vsctl add-port switch1 veth2`

* Se não conseguir ver as interfaces, ver com:
    `sudo ip link`

* Associar ips para as interfaces:
    `sudo ifconfig veth1 10.0.0.5/24`

* Rodar terminal a partir do namespace:
    `sudo ip netns exec VRF1 /bin/bash`

* Ou melhor ainda (igual no mininet):
    `sudo ip netns exec VRF1 xterm`

* Ver largura de banda do enlace (interface):
    `tc qdisk show dev`

* Verificar se as filas foram criadas na interface VETH2:
`sudo ovs-appctl qos/show veth2`


* Checar se as regras de fluxo possuem meter (meters são especificas de versões > 1.3):
`sudo ovs-ofctl -OOpenFlow13 dump-flows switch`

* Outra forma de ver as regras de fluxo:
`sudo ovs-appctl dpif/dump-flows switch`

